In [1]:
using PyCall

INFO: Precompiling module PyCall.
ERROR: LoadError: LoadError: PyCall not properly installed. Please run Pkg.build("PyCall")
 in macro expansion; at C:\Users\Salomon Wollenstein\.julia\v0.5\PyCall\src\startup.jl:44 [inlined]
 in anonymous at .\<missing>:?
 in include_from_node1(::String) at .\loading.jl:488 (repeats 2 times)
 in macro expansion; at .\none:2 [inlined]
 in anonymous at .\<missing>:?
 in eval(::Module, ::Any) at .\boot.jl:234
 in process_options(::Base.JLOptions) at .\client.jl:242
 in _start() at .\client.jl:321
while loading C:\Users\Salomon Wollenstein\.julia\v0.5\PyCall\src\startup.jl, in expression starting on line 41
while loading C:\Users\Salomon Wollenstein\.julia\v0.5\PyCall\src\PyCall.jl, in expression starting on line 39


LoadError: Failed to precompile PyCall to C:\Users\Salomon Wollenstein\.julia\lib\v0.5\PyCall.ji.

In [ ]:
string("pwd()", "functions.py")

In [ ]:
Pkg.update()

In [ ]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport ODandRouteChoiceMat

In [ ]:
Pkg.pin("MacroTools", v"0.3.4")

In [ ]:
xi_list = GLS_Apr_weekday_AM.xi_list
P = GLS_Apr_weekday_AM.P
L = GLS_Apr_weekday_AM.L  # dimension of xi

size(P, 1), size(P, 2)

using JuMP
using Ipopt

mGLSJulia = Model(solver=IpoptSolver())

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum(p[i,j] for j = 1:size(P,2)) == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum(p[i,l] * lam[i] for i = 1:size(P,1)) == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum(p[1,j] for j = 1:size(P,2)))  # play no actual role, but could not use zero objective

solve(mGLSJulia)

getvalue(lam)

getObjectiveValue(mGLSJulia)

GLS_Apr_weekday_AM.saveDemandVec(getvalue(lam))

getvalue(p)

route_choice_matrix_Apr_AM = getvalue(p)

sum(route_choice_matrix_Apr_AM[1, :])

route_choice_matrix_dict_Apr_AM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_AM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_AM)[2]
        route_choice_matrix_dict_Apr_AM[(row_idx, col_idx)] = route_choice_matrix_Apr_AM[row_idx, col_idx]
    end
end

route_choice_matrix_dict_Apr_AM

outfile = open("../temp_files/route_choice_CDC_Apr_AM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_AM)

close(outfile)